# Predictive Modeling Challenge

**Mark Wilber**

The challenge here is to build a classifier for 56 FDA food safety violation categories, which are very unbalanced (sizes spanning more than 3 orders of magnitude). There are two components/features:

* a boolean, `FDAISCRITICAL`, indicating whether the violation is 'critical' or not
* a description of the violation, `VIOCOMMENT`, which can range from 0 to 844 'words'
  * (It is shown below, that the two instances with no comments can be safely dropped.)

This notebook generates TF-IDF features after extracting unigrams and bigrams, and trains models using logistic regression, random forest, linear SVC and complement Naive Bayes to compare f1 scores and training times.

<font color='darkgreen'>**As thise notebook is lengthy, readers will find it much easier to navigate with [Jupyter Nbextensions](https://github.com/ipython-contrib/jupyter_contrib_nbextensions) installed, and Table of Contents (2) selected:**</font>

## Preliminaries

**Next two lines are useful in the event of external code changes.**

In [ ]:
%load_ext autoreload
%autoreload 2

### Python imports

**Next two lines are for pretty output for all prints in a Pandas cell, not just the last.**

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**`DataSci` contains generally helpful data science stuff, while `plotHelpers` includes plot functions specifically.**

In [ ]:
import sys
sys.path.append('/home/wilber/work/Mlib')
from utility import DataSci as util
import plotHelpers as ph

In [ ]:
from time import time, asctime, gmtime
print(asctime(gmtime()))

t0 = time()

# from platform import node
import os
from os.path import exists
# import shutil
# from glob import glob
from random import random
from collections import Counter, OrderedDict
import gc		# garbage collection module
import pprint
# import pickle
import timeit

print("Python version: ", sys.version_info[:])
print("Un-versioned imports:\n")
if 'sys' in sys.modules:
    print('sys', end="")
if 'utility' in sys.modules:
    print(', utility', end="")
if 'plotHelpers' in sys.modules:
    print(', plotHelpers', end="")
if 'platform' in sys.modules:
    print(', platform', end="")
if 'os' in sys.modules:
    print(', os', end="")
if 'os.path' in sys.modules:
    print(', os.path', end="")
if 'shutil' in sys.modules:
    print(', shutil', end="")
if 'glob' in sys.modules:
    print(', glob', end="")
if 'random' in sys.modules:
    print(', random', end="")
if 'collections' in sys.modules:
    print(', collections', end="")
if 'gc' in sys.modules:
    print(', gc', end="")
if 'pprint' in sys.modules:
    print(', pprint', end="")
if 'pickle' in sys.modules:
    print(', pickle', end="")
if 'timeit' in sys.modules:
    print(', timeit', end="")

duVersion = None
from dateutil import __version__ as duVersion
from dateutil.parser import parse
import numpy as np
import pandas as pd
import pyreadr

scVersion = None
from scipy import __version__ as scVersion
import scipy.sparse as sp

skVersion = None
from sklearn import __version__ as skVersion
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import ComplementNB

jlVersion = None
from joblib import __version__ as jlVersion
from joblib import dump, load

import seaborn as sns
import colorcet as cc

mpVersion = None
from matplotlib import __version__ as mpVersion
import matplotlib.pyplot as plt

print("\n")
if 'dateutil' in sys.modules:
    print(f"dateutil: {duVersion}", end="\t")
if 'numpy' in sys.modules:
    print(f"numpy: {np.__version__}", end="\t")
if 'pandas' in sys.modules:
    print(f"pandas: {pd.__version__}", end="\t")
if 'pyreader' in sys.modules:
    print(f"pyreader: {pyreader.__version__}", end="\t")
if 'scipy' in sys.modules:
    print(f"scipy: {scVersion}", end="\t")
# if 'tensorflow' in sys.modules:
#     print(f"tensorflow: {tfVersion}", end="\t")
# if 'keras' in sys.modules:
#     print(f"keras: {kerVersion}", end="\t")
if 'sklearn' in sys.modules:
    print(f"sklearn: {skVersion}", end="\t")
if 'joblib' in sys.modules:
    print(f"joblib: {jlVersion}", end="\t")
if 'seaborn' in sys.modules:
    print(f"seaborn: {sns.__version__}", end="\t")
if 'colorcet' in sys.modules:
    print(f"colorcet: {cc.__version__}", end="\t")
if 'matplotlib' in sys.modules:
    print(f"matplotlib: {mpVersion}", end="\t")
# if '' in sys.modules:
#     print(f": {.__version__}", end="\t")
Δt = time() - t0
print(f"\n\nΔt: {Δt: 4.1f}s.")

%matplotlib inline

## Handle the data

### Read data into a DataFrame

* Have a very quick look at DataFrame characteristics

In [ ]:
fname = "SelectedInspectionReportData.rds"

t0 = time()
result = pyreadr.read_r(fname)
df = result[None]
df.fda_q_fixed = df.fda_q_fixed.astype('int')
df.FDAISCRITICAL = df.FDAISCRITICAL.astype('int')
Δt = time() - t0
print(f"\n\nΔt: {Δt: 4.1f}s.")

df.shape

In [ ]:
df.head(6).T
df.tail(6).T

#### Basic summary

In [ ]:
df.info()

In [ ]:
df.describe()

#### Remove columns from DataFrame which we won't need

In [ ]:
df = df[['fda_q_fixed', 'VIOCOMMENT', 'FDAISCRITICAL']]
df.info()

### Exploratory analysis

#### Classes and relative balance

* The stuff using patches is for placing counts above each rectangle in the bar plot

In [ ]:
FDAcodes = list(set(df['fda_q_fixed'].values))
print(FDAcodes)
classCts = pd.DataFrame(df['fda_q_fixed'].value_counts())
with pd.option_context("display.max_columns", 60):
    display(classCts.T)

In [ ]:
ph.plotValueCounts(df, 'fda_q_fixed', titleText='FDA code frequencies', saveAs='svg', ylim=[0.0, 187500.0])

***The class sizes span nearly 4 orders of magnitude!***

#### Word frequencies

In [ ]:
t0 = time()
df['commentsWords'] = df['VIOCOMMENT'].apply(lambda s: s.split())
t1 = time()
Δt = t1 - t0
print(f"Δt: {Δt % 60.0:4.1f}s.")

In [ ]:
comments = list(df['commentsWords'])

##### Distribution of comment lengths

* Add length of each comment to DataFrame as `wordFreq` column

In [ ]:
wordLens = [len(wordList) for wordList in comments]
df['wordFreq'] = wordLens
wordFreqMode = df['wordFreq'].mode().values[0]

wordCtSorted = sorted(wordLens)
print("smallest word counts:\n", wordCtSorted[:100])
print("largest word counts:\n", wordCtSorted[-101:-1])

**Detailed histogram**

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(18, 3.5))

ph.detailedHistogram(wordLens, ylabel='frequency', volubility=2,
                     titleText=f"Word counts (max: {wordCtSorted[-1]}, mode: {wordFreqMode})",
                     figName="WordCountsHist", ax=ax, ylim = [0.5, 100000.0], ylog=True, saveAs='svg')

In [ ]:
# fig, ax = plt.subplots(1, 1, figsize=(18, 3.5))
# ax.hist(wordLens, bins=np.linspace(0, 845, 846))
# ax.set_yscale('log')
# ax.set_ylim([0.5, 100000.0])
# ax.set_ylabel('freqency')
# plt.title(f"Word counts (max: {wordCtSorted[-1]}, mode: {wordFreqMode})")
# plt.tight_layout(rect=[0, 0.03, 1, 0.975])
# plt.savefig("WordCountsHist.svg")

**Make space**

In [ ]:
del wordLens
del wordCtSorted
del df['commentsWords']

##### What FDA codes correspond to those comments having `wordFreq== 0`?

In [ ]:
df[df['wordFreq']==0]
print("\n", df.shape)

**Can safely remove a couple of records from the 2nd-most populated category**

* Originally there were 1307986 records in `df`, out of which 122314 were in Class 49

In [ ]:
df = df[df['wordFreq']!=0]
df.shape

##### `wordFreq` percentiles

* These show that would get 99% coverage of the comments without truncation if were to use, say, 140-element LSTMs

In [ ]:
df.describe(percentiles=[0.01, 0.05, 0.15, 0.25, 0.5, 0.75, 0.85, 0.95, 0.99])

#### Most-common words

In [ ]:
allWords = [word for wordList in comments for word in wordList]		# Flatten list of lists of words
print(len(comments), len(allWords))

print(comments[:5], "\n", allWords[:25])

In [ ]:
t0 = time()
wordCtr = Counter(allWords)
t1 = time()
Δt = t1 - t0
print(f"Δt: {Δt % 60.0:4.1f}s.")

##### Most common words, after removing stop words

*Result looks very plausible*

In [ ]:
stopWords = text.ENGLISH_STOP_WORDS.union(['-'])

wcStops = [k for k in wordCtr if k.lower() in stopWords]
for k in wcStops:
    del wordCtr[k]
wordCtr.most_common(40)

#### Clean up

In [ ]:
del allWords
del wordCtr

#### `fda_q_fixed` vs. `FDAISCRITICAL`

What is the relationship between the critical violation boolean and the FDA code?

In [ ]:
dfCrit = df.groupby(['fda_q_fixed', 'FDAISCRITICAL']).count()
del dfCrit['VIOCOMMENT']
del dfCrit['wordFreq']
dfCrit.head(20)

dfCrit.reset_index(inplace=True)
dfCrit.head(20)

In [ ]:
plt.rcParams['xtick.top'] = True
plt.rcParams['xtick.labeltop'] = True

fig, ax = plt.subplots(1, 1, figsize=(12, 4))
dfCrit.plot.scatter('fda_q_fixed', 'FDAISCRITICAL', s=4, c='black', ax=ax)
for xv in np.linspace(0.5, 56.5, 57):
    _ = plt.axvline(x=xv, c="#FFB0FF", linewidth=1)
plt.suptitle('Critical violations vs FDA code')
ax.set_xlim([0.5, 56.5])
plt.tight_layout(rect=[0, 0.03, 1, 0.97])
plt.savefig('CriticalViolationVsFDAcode.svg')

**The critical violations plot shows that `FDAISCRITICAL` should be predictive (and certainly should be included in the model):**

* **<font color="darkgreen">classes 30, 32, 34 &amp; 46 *never* have critical violations</font>**
* **<font color="darkgreen">classes 7, 26, 27 &amp; 29 *only* have critical violations</font>**

## Pre-process data

* Here we use TF-IDF features to represent the comment text.

### Split DataFrame by classes

* create a `numpy.random.RandomState` instance to keep track of the random number initializer, in order to ensure consistent results throughout

`splitDataFrameByClasses()` will create two new DataFrames, dfTr, dfTe, according to the desired splits.

<font color='darkgreen'><b>Note that if you just want to do stratified sampling on a numpy array of</b> `X` <b>values,</b> `splitDataFramByClasses()` <b>is not needed.</b> `train_test_split()` <b>accepts the keyword</b> `stratify=myTargetVariable`.</b></font>

* Splitting is done on a per-class basis, so that random selection will not, by chance, yield huge imbalances in train-test splits of tiny classes

In [ ]:
randomState=0
myRandomState = np.random.RandomState(randomState)

In [ ]:
classColumn = 'fda_q_fixed'
dfTr, dfTe = util.splitDataFrameByClasses(df, classColumn,
                                          testFrac=0.50,
                                          myRandomState=myRandomState)
dfTr.shape, dfTe.shape
dfTr.head()
dfTe.head()

**As intended, `splitBalancedDataFrameClasses()` created a new train DataFrame with 10000 $\times$ 56 = 560000 rows, and a test DataFrame ~ 1307984/2 = 653992 rows.**

*The test DataFrame is not exactly an even split of the original, since the splitting is done by class and unioned.*

### TF-IDF

Let's compute TF-IDFs for each comment using scikit-learn's TF-IDF vectorizer.

* We'll use a logarithmic form: `sublinear_df=True`
* Ensure Euclidian norms for each vecture: `norm='l2'`
* Include bigrams, as well as unigrams: `ngram_range=(1,2)`
* English stop words: `stop_words='english'`
* toss very infrequent terms (appearing in fewer than 5 comments): `min_df=5`

#### Fit TF-IDF  features; transform train set (~40 seconds)

In [ ]:
dfTe[['FDAISCRITICAL']].head(16).T

In [ ]:
t0 = time()
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2',
                        encoding='latin-1', ngram_range=(1, 2),
                        stop_words='english')
Xtr = tfidf.fit_transform(dfTr.VIOCOMMENT)
Xtr.shape
yTr = dfTr.fda_q_fixed
t1 = time()
Δt = t1 - t0
print(f"Δt: {Δt % 60.0:4.1f}s.")

**Xtr has 653979 ~ 653992 = 1307984 / 2 rows and ~196k columns. The latter represent the ~196k features in the training set unigram and bigram 'vocabulary'**

In [ ]:
type(Xtr)
for i, row in enumerate(Xtr):
    if i > 1:
        break
    print(i, "\n", row)
    i += 1

#### Transform test set (~35 seconds)

In [ ]:
t0 = time()
Xte = tfidf.transform(dfTe.VIOCOMMENT)
Xte.shape
yTe = dfTe.fda_q_fixed
t1 = time()
Δt = t1 - t0
print(f"Δt: {Δt % 60.0:4.1f}s.")

**same number of columns for Xte, as for Xtr.**

#### Sanity check: show features most correlated with classes

* Figure out which of 196k features are most correlated with each of the FDA codes (~1 minute, 10 seconds)

In [ ]:
t0 = time()
N = 5

FDAcodes = sorted(list(set(df.fda_q_fixed)))

for FDAcode in FDAcodes:
    Xχ2 = chi2(Xtr, yTr==FDAcode)
    indices = np.argsort(Xχ2[0])
    featureNames = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in featureNames if len(v.split(' ')) == 1]
    bigrams = [v for v in featureNames if len(v.split(' ')) == 2]
    print(f"\nFCA code {FDAcode:02d}:")
    print("Most correlated unigrams::\t{{{}".format('}  {'.join(unigrams[-N:])), end='}')
    print("\nMost correlated bigrams::\t{{{}".format('}  {'.join(bigrams[-N:])), end='}')
t1 = time()
Δt = t1 - t0
print(f"\n\nΔt: {int(Δt//60)}m, {Δt % 60.0:4.1f}s.")

#### Add `FDAISCRITICAL` column vales to `Xtr` and `Xte`

In [ ]:
Xtrp = sp.hstack([Xtr, sp.csr_matrix(dfTr[['FDAISCRITICAL']])], 'csr')
np.shape(Xtrp)

In [ ]:
Xtep = sp.hstack([Xte, sp.csr_matrix(dfTe[['FDAISCRITICAL']])], 'csr')
np.shape(Xtep)

In [ ]:
del Xtr
del Xte

### Do some garbage collection before building models

In [ ]:
for i in range(2):
    print('\nCollecting {} ...'.format(i))
    n = gc.collect()
    print('Unreachable objects:', n)
    print('Remaining Garbage:', end=' ')
    pprint.pprint(gc.garbage)

## Models

### Logistic Regression

#### TF-IDF with `FDAISCRITICAL`

* Run defaults for now, except `class_weight='balanced'`
* `n_jobs=7` means use 8 processors (as it is set to $n_{\mathrm{threads}} - 1$)
* For np.shape(Xtrp) = (560000, 196227), fit takes ~30 min.

In [ ]:
np.shape(Xtrp), np.shape(yTr), np.shape(Xtep), np.shape(yTe)

In [ ]:
print(asctime(gmtime()))
LRname = 'LogisticRegressor0.joblib'

if os.path.isfile(LRname):
    LR = load(LRname)
    t1 = time()
else:
    CV = 5
    LR = LogisticRegression(random_state=myRandomState, n_jobs=7,
                            solver='newton-cg', class_weight='balanced')
    t0 = time()
    LR.fit(Xtrp, yTr)
    t1 = time()
    Δt01 = t1 - t0
    print("Δt01: {0} m, {1:4.1f} s.".format(int(Δt01//60), Δt01 % 60.0))

yPred = LR.predict(Xtep)
t2 = time()
Δt12 = t2 - t1
print("Δt12: {0} m, {1:4.1f} s.".format(int(Δt12//60), Δt12 % 60.0))

In [ ]:
print(yTe[:20].values)
print(yPred[:20])

**Save model to disk**

In [ ]:
if not os.path.isfile(LRname):
    dump(LR, LRname)

#### Overall accuracy, precision, recall

In [ ]:
confusionMat = confusion_matrix(yTe, yPred)
print(confusionMat)

In [ ]:
accuracy = np.trace(confusionMat)/np.sum(confusionMat)
recall = np.diag(confusionMat)/np.sum(confusionMat, axis=1)
precision = np.diag(confusionMat)/np.sum(confusionMat, axis=0)
print(f"accuracy: {accuracy:0.3f}, "
      f"<precision>: {np.mean(precision):0.3f}, "
      f"<recall>: {np.mean(recall):0.3f}")

##### Recall, precision by class

Note:

* `macro avg`: $\frac{1}{K}\sum_k m_k$, where $K$ is count of classes and $m_k$ is a given metric for class $k$
* `weighted avg`: $\frac{1}{N}\sum_k n_k \cdot m_k$, where $N$ is count of data instance, $n_k$ is the count of points in class $k$ and $m_k$ is a given metric for class $k$.

In [ ]:
print(metrics.classification_report(yTe.values, yPred, target_names=[str(c)for c in FDAcodes]))

In [ ]:
classCts = dfTe['fda_q_fixed'].value_counts()

recall = np.diag(confusionMat)/np.sum(confusionMat, axis = 1)
precision = np.diag(confusionMat)/np.sum(confusionMat, axis = 0)
f1 = 2.0*precision*recall/(precision + recall)
print("class\tprecision\trecall\tf1\tsize")

for FDAcode, classCt in classCts.iteritems():
    print(f"{FDAcode}\t{precision[FDAcode - 1]:0.3f}\t\t{recall[FDAcode - 1]:0.3f}\t{f1[FDAcode - 1]:0.3f}\t\t{classCt:d}")

##### Plot confusion matrix

* As this is a straight confusion matrix, diagonal elements mostly reflect class size in test set
* *This is hard to interpret by visual inspection alone*

In [ ]:
labelFontSz = 16
tickFontSz = 13
titleFontSz = 20

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(25, 25))
ph.plotConfusionMatrix(confusionMat, saveAs='pdf', xlabels=FDAcodes,
                       ylabels=FDAcodes, titleText = 'Logistic Regression',
                       ax = ax,  xlabelFontSz=labelFontSz,
                       ylabelFontSz=labelFontSz, xtickFontSz=tickFontSz,
                       ytickFontSz=tickFontSz, titleFontSz=titleFontSz)

##### Plot recall confusion matrix (normalized by row)

* diagonal elements now represent the *recall* for each class

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(25, 25))
ph.plotConfusionMatrix(confusionMat, saveAs='pdf', xlabels=FDAcodes, type='recall',
                       ylabels=FDAcodes, titleText = 'Logistic Regression',
                       ax = ax,  xlabelFontSz=labelFontSz,
                       ylabelFontSz=labelFontSz, xtickFontSz=tickFontSz,
                       ytickFontSz=tickFontSz, titleFontSz=titleFontSz)

##### Plot precision confusion matrix (normalized by column)

* diagonal elements now represent the *precision* for each class

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(25, 25))
ph.plotConfusionMatrix(confusionMat, saveAs='pdf', xlabels=FDAcodes, type='precision',
                       ylabels=FDAcodes, titleText = 'Logistic Regression',
                       ax = ax,  xlabelFontSz=labelFontSz,
                       ylabelFontSz=labelFontSz, xtickFontSz=tickFontSz,
                       ytickFontSz=tickFontSz, titleFontSz=titleFontSz)

In [ ]:
del LR

#### Do some garbage collection before building RF models

In [ ]:
for i in range(2):
    print('\nCollecting {} ...'.format(i))
    n = gc.collect()
    print('Unreachable objects:', n)
    print('Remaining Garbage:', end=' ')
    pprint.pprint(gc.garbage)

### Random Forest

#### TF-IDF with `FDAISCRITICAL`, `max_depth=250`

* Run defaults for now, except `n_estimators=160` (default is 10)
* For np.shape(Xtr) = (560000, 178558), fit takes ~150 min.

<font color='darkred'>***The very long training time in this instance is due to memory swapping.
(Fortunately, on my 12 GB machine, the swap space is on a flash drive.)
Reducing the `max_depth` from 250 should help to reduce the memory requirements.***</font>

In [ ]:
np.shape(Xtrp), np.shape(yTr), np.shape(Xtep), np.shape(yTe)

In [ ]:
print(asctime(gmtime()))
RFname = 'RandomForest0.joblib'

if os.path.isfile(RFname):
    RF = load(RFname)
    t1 = time()
else:
    RF = RandomForestClassifier(random_state=myRandomState, n_jobs=7, n_estimators=160,
                                max_depth=250, class_weight='balanced')
    t0 = time()
    RF.fit(Xtrp, yTr)
    t1 = time()
    Δt01 = t1 - t0
    print("Δt01: {0} m, {1:4.1f} s.".format(int(Δt01//60), Δt01 % 60.0))

yPred = RF.predict(Xtep)
t2 = time()
Δt12 = t2 - t1
print("Δt12: {0} m, {1:4.1f} s.".format(int(Δt12//60), Δt12 % 60.0))

In [ ]:
print(yTe[:20].values)
print(yPred[:20])

**Save model to disk**

<font color="darkred">Nope. Not agonna do this; model is yuge &mdash; 15 GB!</font>

In [ ]:
# if not os.path.isfile(RFname):
#     dump(RF, RFname)

#### Overall accuracy, precision, recall

In [ ]:
confusionMat = confusion_matrix(yTe, yPred)
print(confusionMat)

In [ ]:
accuracy = np.trace(confusionMat)/np.sum(confusionMat)
recall = np.diag(confusionMat)/np.sum(confusionMat, axis=1)
precision = np.diag(confusionMat)/np.sum(confusionMat, axis=0)
print(f"accuracy: {accuracy:0.3f}, "
      f"<precision>: {np.mean(precision):0.3f}, "
      f"<recall>: {np.mean(recall):0.3f}")

##### Recall, precision by class

In [ ]:
print(metrics.classification_report(yTe.values, yPred, target_names=[str(c)for c in FDAcodes]))

In [ ]:
classCts = dfTe['fda_q_fixed'].value_counts()

recall = np.diag(confusionMat)/np.sum(confusionMat, axis = 1)
precision = np.diag(confusionMat)/np.sum(confusionMat, axis = 0)
f1 = 2.0*precision*recall/(precision + recall)
print("class\tprecision\trecall\tf1\tsize")

for FDAcode, classCt in classCts.iteritems():
    print(f"{FDAcode}\t{precision[FDAcode - 1]:0.3f}\t\t{recall[FDAcode - 1]:0.3f}\t{f1[FDAcode - 1]:0.3f}\t\t{classCt:d}")

##### Plot confusion matrix

* As this is a straight confusion matrix, diagonal elements mostly reflect class size in test set
* *This is hard to interpret by visual inspection alone*

In [ ]:
labelFontSz = 16
tickFontSz = 13
titleFontSz = 20

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(25, 25))
ph.plotConfusionMatrix(confusionMat, saveAs='pdf', xlabels=FDAcodes,
                       ylabels=FDAcodes, titleText = 'Random Forest (160 estimators, max depth=250)',
                       ax = ax,  xlabelFontSz=labelFontSz,
                       ylabelFontSz=labelFontSz, xtickFontSz=tickFontSz,
                       ytickFontSz=tickFontSz, titleFontSz=titleFontSz)

##### Plot recall confusion matrix (normalized by row)

* diagonal elements now represent the *recall* for each class

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(25, 25))
ph.plotConfusionMatrix(confusionMat, saveAs='pdf', xlabels=FDAcodes, type='recall',
                       ylabels=FDAcodes, titleText = 'Random Forest (160 estimators, max depth=250)',
                       ax = ax,  xlabelFontSz=labelFontSz,
                       ylabelFontSz=labelFontSz, xtickFontSz=tickFontSz,
                       ytickFontSz=tickFontSz, titleFontSz=titleFontSz)

##### Plot precision confusion matrix (normalized by column)

* diagonal elements now represent the *precision* for each class

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(25, 25))
ph.plotConfusionMatrix(confusionMat, saveAs='pdf', xlabels=FDAcodes, type='precision',
                       ylabels=FDAcodes, titleText = 'Random Forest (160 estimators, max depth=250)',
                       ax = ax,  xlabelFontSz=labelFontSz,
                       ylabelFontSz=labelFontSz, xtickFontSz=tickFontSz,
                       ytickFontSz=tickFontSz, titleFontSz=titleFontSz)

In [ ]:
del RF

#### Do some garbage collection before building RF models

In [ ]:
for i in range(2):
    print('\nCollecting {} ...'.format(i))
    n = gc.collect()
    print('Unreachable objects:', n)
    print('Remaining Garbage:', end=' ')
    pprint.pprint(gc.garbage)

### LinearSVC

#### TF-IDF with `FDAISCRITICAL`

* `penalty='l1', dual=False, class_weight='balanced', random_state=myRandomState
* For np.shape(Xtr) = (560000, 178558), fit takes ~45 min.

In [ ]:
np.shape(Xtrp), np.shape(yTr), np.shape(Xtep), np.shape(yTe)

In [ ]:
print(asctime(gmtime()))
LSVCname = 'LinearSVC0.joblib'

if os.path.isfile(LSVCname):
    RF = load(RFname)
    t1 = time()
else:
    LSVC = LinearSVC(random_state=myRandomState, penalty='l1',
                     dual=False, class_weight='balanced')
    t0 = time()
    LSVC.fit(Xtrp, yTr)
    t1 = time()
    Δt01 = t1 - t0
    print("Δt01: {0} m, {1:4.1f} s.".format(int(Δt01//60), Δt01 % 60.0))

yPred = LSVC.predict(Xtep)
t2 = time()
Δt = t2 - t1
print(f"\n\nΔt: {int(Δt//60)}m, {Δt % 60.0:4.1f}s.")

In [ ]:
print(yTe[:20].values)
print(yPred[:20])

**Save model to disk**

In [ ]:
if not os.path.isfile(LSVCname):
    dump(LSVC, LSVCname)

#### Overall accuracy, precision, recall

In [ ]:
confusionMat = confusion_matrix(yTe, yPred)
print(confusionMat)

In [ ]:
accuracy = np.trace(confusionMat)/np.sum(confusionMat)
recall = np.diag(confusionMat)/np.sum(confusionMat, axis=1)
precision = np.diag(confusionMat)/np.sum(confusionMat, axis=0)
print(f"accuracy: {accuracy:0.3f}, "
      f"<precision>: {np.mean(precision):0.3f}, "
      f"<recall>: {np.mean(recall):0.3f}")

##### Recall, precision by class

In [ ]:
print(metrics.classification_report(yTe.values, yPred, target_names=[str(c)for c in FDAcodes]))

In [ ]:
classCts = dfTe['fda_q_fixed'].value_counts()

recall = np.diag(confusionMat)/np.sum(confusionMat, axis = 1)
precision = np.diag(confusionMat)/np.sum(confusionMat, axis = 0)
f1 = 2.0*precision*recall/(precision + recall)
print("class\tprecision\trecall\tf1\tsize")

for FDAcode, classCt in classCts.iteritems():
    print(f"{FDAcode}\t{precision[FDAcode - 1]:0.3f}\t\t{recall[FDAcode - 1]:0.3f}\t{f1[FDAcode - 1]:0.3f}\t\t{classCt:d}")

##### Plot confusion matrix

* As this is a straight confusion matrix, diagonal elements mostly reflect class size in test set
* *This is hard to interpret by visual inspection alone*

In [ ]:
labelFontSz = 16
tickFontSz = 13
titleFontSz = 20

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(25, 25))
ph.plotConfusionMatrix(confusionMat, saveAs='pdf', xlabels=FDAcodes,
                       ylabels=FDAcodes, titleText = "Linear SVC ('l1' penalty, dual=False)",
                       ax = ax,  xlabelFontSz=labelFontSz,
                       ylabelFontSz=labelFontSz, xtickFontSz=tickFontSz,
                       ytickFontSz=tickFontSz, titleFontSz=titleFontSz)

##### Plot recall confusion matrix (normalized by row)

* diagonal elements now represent the *recall* for each class

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(25, 25))
ph.plotConfusionMatrix(confusionMat, saveAs='pdf', xlabels=FDAcodes, type='recall',
                       ylabels=FDAcodes, titleText = "Linear SVC ('l1' penalty, dual=False)",
                       ax = ax,  xlabelFontSz=labelFontSz,
                       ylabelFontSz=labelFontSz, xtickFontSz=tickFontSz,
                       ytickFontSz=tickFontSz, titleFontSz=titleFontSz)

##### Plot precision confusion matrix (normalized by column)

* diagonal elements now represent the *precision* for each class

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(25, 25))
ph.plotConfusionMatrix(confusionMat, saveAs='pdf', xlabels=FDAcodes, type='precision',
                       ylabels=FDAcodes, titleText = "Linear SVC ('l1' penalty, dual=False)",
                       ax = ax,  xlabelFontSz=labelFontSz,
                       ylabelFontSz=labelFontSz, xtickFontSz=tickFontSz,
                       ytickFontSz=tickFontSz, titleFontSz=titleFontSz)

In [ ]:
del LSVC

#### Do some garbage collection before building RF models

In [ ]:
for i in range(2):
    print('\nCollecting {} ...'.format(i))
    n = gc.collect()
    print('Unreachable objects:', n)
    print('Remaining Garbage:', end=' ')
    pprint.pprint(gc.garbage)

### Complement Naive Bayes

#### TF-IDF with `FDAISCRITICAL`

* default hyperparameters, except `fit_prior=False`, as we have enforced train clasess with ≥ 14 elements
* For np.shape(Xtr) = (560000, 178558), fit takes ~ min.

In [ ]:
np.shape(Xtrp), np.shape(yTr), np.shape(Xtep), np.shape(yTe)

In [ ]:
print(asctime(gmtime()))
CNBname = 'ComplementNaiveBayes0.joblib'

if os.path.isfile(CNBname):
    CNB = load(CNBname)
    t1 = time()
else:
    CNB = ComplementNB(fit_prior=False)
    t0 = time()
    CNB.fit(Xtrp, yTr)
    t1 = time()
    Δt01 = t1 - t0
    print("Δt01: {0} m, {1:4.1f} s.".format(int(Δt01//60), Δt01 % 60.0))

yPred = CNB.predict(Xtep)
t2 = time()
Δt = t2 - t1
print(f"\n\nΔt: {int(Δt//60)}m, {Δt % 60.0:4.1f}s.")

In [ ]:
print(yTe[:20].values)
print(yPred[:20])

**Save model to disk**

In [ ]:
if not os.path.isfile(CNBname):
    dump(CNB, CNBname)

#### Overall accuracy, precision, recall

In [ ]:
confusionMat = confusion_matrix(yTe, yPred)
print(confusionMat)

In [ ]:
accuracy = np.trace(confusionMat)/np.sum(confusionMat)
recall = np.diag(confusionMat)/np.sum(confusionMat, axis=1)
precision = np.diag(confusionMat)/np.sum(confusionMat, axis=0)
print(f"accuracy: {accuracy:0.3f}, "
      f"<precision>: {np.mean(precision):0.3f}, "
      f"<recall>: {np.mean(recall):0.3f}")

##### Recall, precision by class

In [ ]:
print(metrics.classification_report(yTe.values, yPred, target_names=[str(c)for c in FDAcodes]))

In [ ]:
classCts = dfTe['fda_q_fixed'].value_counts()

recall = np.diag(confusionMat)/np.sum(confusionMat, axis = 1)
precision = np.diag(confusionMat)/np.sum(confusionMat, axis = 0)
f1 = 2.0*precision*recall/(precision + recall)
print("class\tprecision\trecall\tf1\tsize")

for FDAcode, classCt in classCts.iteritems():
    print(f"{FDAcode}\t{precision[FDAcode - 1]:0.3f}\t\t{recall[FDAcode - 1]:0.3f}\t{f1[FDAcode - 1]:0.3f}\t\t{classCt:d}")

##### Plot confusion matrix

* As this is a straight confusion matrix, diagonal elements mostly reflect class size in test set
* *This is hard to interpret by visual inspection alone*

In [ ]:
labelFontSz = 16
tickFontSz = 13
titleFontSz = 20

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(25, 25))
ph.plotConfusionMatrix(confusionMat, saveAs='pdf', xlabels=FDAcodes,
                       ylabels=FDAcodes, titleText = 'Complement Naive Bayes',
                       ax = ax,  xlabelFontSz=labelFontSz,
                       ylabelFontSz=labelFontSz, xtickFontSz=tickFontSz,
                       ytickFontSz=tickFontSz, titleFontSz=titleFontSz)

##### Plot recall confusion matrix (normalized by row)

* diagonal elements now represent the *recall* for each class

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(25, 25))
ph.plotConfusionMatrix(confusionMat, saveAs='pdf', xlabels=FDAcodes, type='recall',
                       ylabels=FDAcodes, titleText = 'Complement Naive Bayes',
                       ax = ax,  xlabelFontSz=labelFontSz,
                       ylabelFontSz=labelFontSz, xtickFontSz=tickFontSz,
                       ytickFontSz=tickFontSz, titleFontSz=titleFontSz)

##### Plot precision confusion matrix (normalized by column)

* diagonal elements now represent the *precision* for each class

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(25, 25))
ph.plotConfusionMatrix(confusionMat, saveAs='pdf', xlabels=FDAcodes, type='precision',
                       ylabels=FDAcodes, titleText = 'Complement Naive Bayes',
                       ax = ax,  xlabelFontSz=labelFontSz,
                       ylabelFontSz=labelFontSz, xtickFontSz=tickFontSz,
                       ytickFontSz=tickFontSz, titleFontSz=titleFontSz)